## **Holo1.5-3B (Navigation & Localization)**

Holo1.5 series provides state-of-the-art foundational models for building such agents. Holo1.5 models excel at user interface (UI) localization and UI-based question answering (QA) across web, computer, and mobile environments, with strong performance on benchmarks including Screenspot-V2, Screenspot-Pro, GroundUI-Web, Showdown, and our newly introduced WebClick.

Computer Use (CU) agents are AI systems that can interact with real applications—web, desktop, and mobile—on behalf of a user. They can navigate interfaces, manipulate elements, and answer questions about content, enabling powerful automation and productivity tools. CU agents are becoming increasingly important as they allow humans to delegate complex digital tasks safely and efficiently.

### **Install Packages**

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/transformers.git \
             git+https://github.com/huggingface/accelerate.git \
             git+https://github.com/huggingface/peft.git \
             transformers-stream-generator huggingface_hub albumentations \
             pyvips-binary qwen-vl-utils sentencepiece opencv-python docling-core \
             python-docx torchvision safetensors matplotlib num2words \

!pip install xformers requests pymupdf hf_xet spaces pyvips pillow gradio \
             einops torch fpdf timm av decord bitsandbytes reportlab numpy
#Hold tight, this will take around 1-2 minutes.

### **Run App Demo**

In [ ]:
import os
import re
import traceback
from datetime import datetime
from typing import Any, Literal

import gradio as gr
import numpy as np
import requests
import spaces
import torch
from PIL import Image, ImageDraw
from pydantic import BaseModel, Field
from transformers import AutoProcessor, BitsAndBytesConfig
from transformers.models.auto.modeling_auto import AutoModelForImageTextToText
from transformers.models.qwen2_vl.image_processing_qwen2_vl import smart_resize

# --- Configuration ---
# Define model options
MODEL_OPTIONS = {
    "Holo1.5-3B": "Hcompany/Holo1.5-3B",
}
# Select the model to use
SELECTED_MODEL_NAME = "Holo1.5-3B"
MODEL_ID = MODEL_OPTIONS[SELECTED_MODEL_NAME]


# --- Model and Processor Loading (with 4-bit Quantization) ---
print(f"Loading model and processor for {MODEL_ID} with 4-bit quantization...")
model = None
processor = None
model_loaded = False
load_error_message = ""

try:
    # Define 4-bit quantization configuration
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    # Load the quantized model
    model = AutoModelForImageTextToText.from_pretrained(
        MODEL_ID,
        trust_remote_code=True,
        quantization_config=quantization_config,
        device_map="auto"
    )
    # Load the processor
    processor = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)

    model_loaded = True
    print("Model and processor loaded successfully.")
except Exception as e:
    load_error_message = (
        f"Error loading model/processor: {e}\n"
        "This might be due to network issues, an incorrect model ID, or missing dependencies (like flash_attention_2 if enabled by default in some config).\n"
        "Ensure you have a stable internet connection and the necessary libraries installed (bitsandbytes, accelerate)."
    )
    print(load_error_message)


# --- Common Utility Functions ---
def array_to_image(image_array: np.ndarray) -> Image.Image:
    if image_array is None:
        raise ValueError("No image provided. Please upload an image before submitting.")
    # Convert numpy array to PIL Image
    img = Image.fromarray(np.uint8(image_array))
    return img


# --- Navigation Specific Code ---

SYSTEM_PROMPT: str = """Imagine you are a robot browsing the web, just like humans. Now you need to complete a task.
In each iteration, you will receive an Observation that includes the last  screenshots of a web browser and the current memory of the agent.
You have also information about the step that the agent is trying to achieve to solve the task.
Carefully analyze the visual information to identify what to do, then follow the guidelines to choose the following action.
You should detail your thought (i.e. reasoning steps) before taking the action.
Also detail in the notes field of the action the extracted information relevant to solve the task.
Once you have enough information in the notes to answer the task, return an answer action with the detailed answer in the notes field.
This will be evaluated by an evaluator and should match all the criteria or requirements of the task.
Guidelines:
- store in the notes all the relevant information to solve the task that fulfill the task criteria. Be precise
- Use both the task and the step information to decide what to do
- if you want to write in a text field and the text field already has text, designate the text field by the text it contains and its type
- If there is a cookies notice, always accept all the cookies first
- The observation is the screenshot of the current page and the memory of the agent.
- If you see relevant information on the screenshot to answer the task, add it to the notes field of the action.
- If there is no relevant information on the screenshot to answer the task, add an empty string to the notes field of the action.
- If you see buttons that allow to navigate directly to relevant information, like jump to ... or go to ... , use them to navigate faster.
- In the answer action, give as many details a possible relevant to answering the task.
- if you want to write, don't click before. Directly use the write action
- to write, identify the web element which is type and the text it already contains
- If you want to use a search bar, directly write text in the search bar
- Don't scroll too much. Don't scroll if the number of scrolls is greater than 3
- Don't scroll if you are at the end of the webpage
- Only refresh if you identify a rate limit problem
- If you are looking for a single flights, click on round-trip to select 'one way'
- Never try to login, enter email or password. If there is a need to login, then go back.
- If you are facing a captcha on a website, try to solve it.
- if you have enough information in the screenshot and in the notes to answer the task, return an answer action with the detailed answer in the notes field
- The current date is {timestamp}.
# <output_json_format>
# ```json
# {output_format}
# ```
# </output_json_format>
"""


class ClickElementAction(BaseModel):
    """Click at absolute coordinates of a web element with its description"""
    action: Literal["click_element"] = Field(description="Click at absolute coordinates of a web element")
    element: str = Field(description="text description of the element")
    x: int = Field(description="The x coordinate, number of pixels from the left edge.")
    y: int = Field(description="The y coordinate, number of pixels from the top edge.")

class WriteElementAction(BaseModel):
    """Write content at absolute coordinates of a web element identified by its description, then press Enter."""
    action: Literal["write_element_abs"] = Field(description="Write content at absolute coordinates of a web page")
    content: str = Field(description="Content to write")
    element: str = Field(description="Text description of the element")
    x: int = Field(description="The x coordinate, number of pixels from the left edge.")
    y: int = Field(description="The y coordinate, number of pixels from the top edge.")

class ScrollAction(BaseModel):
    """Scroll action with no required element"""
    action: Literal["scroll"] = Field(description="Scroll the page or a specific element")
    direction: Literal["down", "up", "left", "right"] = Field(description="The direction to scroll in")

class GoBackAction(BaseModel):
    """Action to navigate back in browser history"""
    action: Literal["go_back"] = Field(description="Navigate to the previous page")

class RefreshAction(BaseModel):
    """Action to refresh the current page"""
    action: Literal["refresh"] = Field(description="Refresh the current page")

class GotoAction(BaseModel):
    """Action to go to a particular URL"""
    action: Literal["goto"] = Field(description="Goto a particular URL")
    url: str = Field(description="A url starting with http:// or https://")

class WaitAction(BaseModel):
    """Action to wait for a particular amount of time"""
    action: Literal["wait"] = Field(description="Wait for a particular amount of time")
    seconds: int = Field(default=2, ge=0, le=10, description="The number of seconds to wait")

class RestartAction(BaseModel):
    """Restart the task from the beginning."""
    action: Literal["restart"] = "restart"

class AnswerAction(BaseModel):
    """Return a final answer to the task. This is the last action to call in an episode."""
    action: Literal["answer"] = "answer"
    content: str = Field(description="The answer content")

ActionSpace = ( ClickElementAction | WriteElementAction | ScrollAction | GoBackAction | RefreshAction | WaitAction | RestartAction | AnswerAction | GotoAction)

class NavigationStep(BaseModel):
    note: str = Field(default="", description="Task-relevant information extracted from the previous observation. Keep empty if no new info.")
    thought: str = Field(description="Reasoning about next steps (<4 lines)")
    action: ActionSpace = Field(description="Next action to take")

def get_navigation_prompt(task, image, step=1):
    system_prompt = SYSTEM_PROMPT.format(
        output_format=NavigationStep.model_json_schema(),
        timestamp="2025-06-04 14:16:03",
    )
    return [
        {"role": "system", "content": [{"type": "text", "text": system_prompt},],},
        {"role": "user","content": [
                {"type": "text", "text": f"<task>\n{task}\n</task>\n"},
                {"type": "text", "text": f"<observation step={step}>\n"},
                {"type": "text", "text": "<screenshot>\n"},
                {"type": "image", "image": image,},
                {"type": "text", "text": "\n</screenshot>\n"},
                {"type": "text", "text": "\n</observation>\n"},
            ],},
    ]


@spaces.GPU(duration=20)
def run_inference_navigation(messages_for_template: list[dict[str, Any]], pil_image_for_processing: Image.Image) -> str:
    text_prompt = processor.apply_chat_template(messages_for_template, tokenize=False, add_generation_prompt=True)
    inputs = processor(
        text=[text_prompt],
        images=[pil_image_for_processing],
        padding=True,
        return_tensors="pt",
    ).to(model.device)
    generated_ids = model.generate(**inputs, max_new_tokens=128, do_sample=False)
    generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    decoded_output = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    return decoded_output[0] if decoded_output else ""


def navigate(input_numpy_image: np.ndarray, task: str) -> str:
    if not model_loaded: return f"Model not loaded. Error: {load_error_message}"
    input_pil_image = array_to_image(input_numpy_image)
    image_proc_config = processor.image_processor
    try:
        resized_height, resized_width = smart_resize(
            input_pil_image.height,
            input_pil_image.width,
            factor=image_proc_config.patch_size * image_proc_config.merge_size,
            min_pixels=image_proc_config.min_pixels,
            max_pixels=image_proc_config.max_pixels,
        )
        resized_image = input_pil_image.resize(
            size=(resized_width, resized_height),
            resample=Image.Resampling.LANCZOS,
        )
    except Exception as e:
        return f"Error resizing image: {e}"

    prompt = get_navigation_prompt(task, resized_image, step=1)
    try:
        navigation_str = run_inference_navigation(prompt, resized_image)
    except Exception as e:
        return f"Error during model inference: {e}"
    return navigation_str


# --- Localization Specific Code ---

LOCALIZATION_PROMPT: str = """Localize an element on the GUI image according to the provided target and output a click position.
          * Only output the click position, do not output any other text.
          * The click position should be in the format 'Click(x, y)' with x: num pixels from the left edge and y: num pixels from the top edge
          Your target is:"""


def get_localization_prompt(component, image):
    return [
        {"role": "user", "content": [
                {"type": "image", "image": image,},
                {"type": "text", "text": LOCALIZATION_PROMPT + "\n" + component},
            ],},
    ]


@spaces.GPU
def run_inference_localization(
    messages_for_template: list[dict[str, Any]], pil_image_for_processing: Image.Image
) -> str:
    text_prompt = processor.apply_chat_template(messages_for_template, tokenize=False, add_generation_prompt=True)
    inputs = processor(
        text=[text_prompt],
        images=[pil_image_for_processing],
        padding=True,
        return_tensors="pt",
    ).to(model.device)
    generated_ids = model.generate(**inputs, max_new_tokens=128, do_sample=False)
    generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    decoded_output = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    return decoded_output[0] if decoded_output else ""


def localize(input_numpy_image: np.ndarray, task: str):
    if not model_loaded: return f"Model not loaded. Error: {load_error_message}", None
    input_pil_image = array_to_image(input_numpy_image)
    image_proc_config = processor.image_processor
    try:
        resized_height, resized_width = smart_resize(
            input_pil_image.height,
            input_pil_image.width,
            factor=image_proc_config.patch_size * image_proc_config.merge_size,
            min_pixels=image_proc_config.min_pixels,
            max_pixels=image_proc_config.max_pixels,
        )
        resized_image = input_pil_image.resize(
            size=(resized_width, resized_height),
            resample=Image.Resampling.LANCZOS,
        )
    except Exception as e:
        return f"Error resizing image: {e}", input_pil_image.copy().convert("RGB")

    prompt = get_localization_prompt(task, resized_image)
    try:
        localization = run_inference_localization(prompt, resized_image)
    except Exception as e:
        return f"Error during model inference: {e}", resized_image.copy().convert("RGB")

    output_image_with_click = resized_image.copy().convert("RGB")
    match = re.search(r"Click\((\d+),\s*(\d+)\)", localization)
    if match:
        try:
            x, y = int(match.group(1)), int(match.group(2))
            draw = ImageDraw.Draw(output_image_with_click)
            radius = max(5, min(resized_width // 100, resized_height // 100, 15))
            bbox = (x - radius, y - radius, x + radius, y + radius)
            draw.ellipse(bbox, outline="red", width=max(2, radius // 4))
        except Exception as e:
            traceback.print_exc()
    return localization, output_image_with_click


# --- Gradio UI ---

with gr.Blocks(theme="bethecloud/storj_theme") as demo:
    gr.Markdown(f"<h1 style='text-align: center;'>{SELECTED_MODEL_NAME}: VLM Demo</h1>")

    with gr.Tabs():
        with gr.TabItem("Navigation"):
            with gr.Row():
                with gr.Column():
                    nav_input_image = gr.Image(label="Input UI Image", height=400)
                    nav_task = gr.Textbox(
                        label="Task",
                        placeholder="e.g., Find the latest model by H Company",
                        info="Type the task you want the model to complete.",
                    )
                    nav_button = gr.Button("Navigate", variant="primary")
                with gr.Column():
                    nav_output = gr.Textbox(label="Navigation Step")

            example_image_url_nav = "https://huggingface.co/spaces/Hcompany/Holo1.5-Navigation/resolve/main/desktop_1.png"
            example_image_nav = Image.open(requests.get(example_image_url_nav, stream=True).raw)
            gr.Examples(
                examples=[[example_image_nav, "Find the latest model by H Company"]],
                inputs=[nav_input_image, nav_task],
                outputs=[nav_output],
                fn=navigate,
                cache_examples="lazy",
            )

        with gr.TabItem("Localization"):
            with gr.Row():
                with gr.Column():
                    loc_input_image = gr.Image(label="Input UI Image", height=400)
                    loc_task = gr.Textbox(
                        label="Component",
                        placeholder="Email quote for Hyundai Kona",
                        info="Describe the UI component to find.",
                    )
                    loc_button = gr.Button("Localize", variant="primary")
                with gr.Column():
                    loc_output_coords = gr.Textbox(label="Localization Step")
                    loc_output_image = gr.Image(
                        type="pil", label="Image with coordinates of the component", height=400, interactive=False
                    )

            example_image_url_loc = "https://huggingface.co/spaces/Hcompany/Holo1.5-Localization/resolve/main/desktop_3.png"
            example_image_loc = Image.open(requests.get(example_image_url_loc, stream=True).raw)
            gr.Examples(
                examples=[[example_image_loc, "Email quote for Hyundai Kona"]],
                inputs=[loc_input_image, loc_task],
                outputs=[loc_output_coords, loc_output_image],
                fn=localize,
                cache_examples="lazy",
            )

    nav_button.click(navigate, [nav_input_image, nav_task], [nav_output])
    loc_button.click(localize, [loc_input_image, loc_task], [loc_output_coords, loc_output_image])

demo.queue(api_open=False)
demo.launch(debug=True)